In [ ]:
import numpy as np
import scipy.signal
from psyacmodel import *
import matplotlib.pyplot as plt
import scipy.fftpack as spfft
import librosa

In [ ]:
%matplotlib inline

In [ ]:
def load_cd_quality_audio(filename):
    audio, sr = librosa.load(filename, sr=44100, dtype='float_')
#     max_int_value = 2**15 - 1
#     audio *= max_int_value
#     audio = audio.astype('int16')
    return audio
    
x = load_cd_quality_audio("taxman.wav")

In [ ]:
N = 1152 #number of MDCT subbands
nfilts = 18  #number of subbands in the bark domain
fs = 44100
fb = np.sin(np.pi/(2*N)*(np.arange(int(1.5*N))+0.5))

In [ ]:
maxfreq=fs/2
alpha=0.8  #Exponent for non-linear superposition of spreading functions
nfft=2*N  #number of fft subbands

In [ ]:
W=mapping2barkmat(fs,nfilts,nfft)
W.shape

In [ ]:
W_inv=mappingfrombarkmat(W,nfft)
W_inv.shape
W_inv

In [ ]:
spreadingfunctionBarkdB=f_SP_dB(maxfreq,nfilts)
spreadingfuncmatrix=spreadingfunctionmat(spreadingfunctionBarkdB,alpha,nfilts)

In [ ]:
def symFmatrix(f):
    """produces a diamond shaped folding matrix F from the coefficients f
    (f is a 1-d array)
    which leads to identical analysis and synthesis baseband impulse responses
    Hence has det 1 or -1
    If N is number of subbands, then f is a vector of size 1.5*N coefficients.
    N is even
    returns: F of shape (N,N,1)
    """   
    sym=1.0; #The kind of symmetry: +-1
    N = int(len(f)/1.5);
    F=np.zeros((N,N,1))
    F[0:int(N/2),0:int(N/2),0]=np.fliplr(np.diag(f[0:int(N/2)]))
    F[int(N/2):N,0:int(N/2),0]=np.diag(f[int(N/2):N])
    F[0:int(N/2),int(N/2):N,0]=np.diag(f[N:(N+int(N/2))])
    ff = np.flipud((sym*np.ones((int(N/2))) - (f[N:(int(1.5*N))])*f[N-1:int(N/2)-1:-1])/f[0:int(N/2)])     
    F[int(N/2):N,int(N/2):N,0]=-np.fliplr(np.diag(ff))
    return F

def Dmatrix(N):
    """produces a delay matrix D(z), which has delay z^-1 on the upper half of its diagonal
    in a 3D polynomial representation (exponents of z^-1 are in the third dimension) 
    N is number of subbands and size of the polynomial matrix (NxN) 
    N is even""" 
    D=np.zeros((N,N,2));    
    D[:,:,0] = np.diag(np.append(np.zeros((1,int(N/2))), np.ones((1,int(N/2)))));
    D[:,:,1] = np.diag(np.append(np.ones((1,int(N/2))), np.zeros((1,int(N/2)))));  
    return D;

def x2polyphase(x,N):
    """Converts input signal x (a 1D array) into a polyphase row vector 
    xp for blocks of length N, with shape: (1,N,#of blocks)"""     
    import numpy as np
    #Convert stream x into a 2d array where each row is a block:
    #xp.shape : (y,x, #num blocks):
    x=x[:int(len(x)/N)*N] #limit signal to integer multiples of N
    xp=np.reshape(x,(N,-1),order='F') #order=F: first index changes fastest
    #add 0'th dimension for function polmatmult:
    xp=np.expand_dims(xp,axis=0)
    return xp


#The DCT4 transform:
def DCT4(samples):
   #Argument: 3-D array of samples, shape (y,x,# of blocks), each row correspond to 1 row 
   #to apply the DCT to.
   #Output: 3-D array where each row ist DCT4 transformed, orthonormal.
   import numpy as np
   #use a DCT3 to implement a DCT4:
   r,N,blocks=samples.shape
   samplesup=np.zeros((1,2*N,blocks))
   #upsample signal:
   samplesup[0,1::2,:]=samples
   y=spfft.dct(samplesup,type=3,axis=1,norm='ortho')*np.sqrt(2)
   return y[:,0:N,:]

def polmatmult( A,B ):
   """polmatmult(A,B)
   multiplies two polynomial matrices (arrays) A and B, where each matrix entry is a polynomial.
   Those polynomial entries are in the 3rd dimension
   The thirs dimension can also be interpreted as containing the (2D) coefficient
   exponent of z^-1.
   Result is C=A*B;"""  
   [NAx, NAy, NAz] = np.shape(A);
   [NBx, NBy, NBz] = np.shape(B);
   #Degree +1 of resulting polynomial, with NAz-1 and NBz-1 being the degree of the...
   Deg = NAz + NBz -1;
   C = np.zeros((NAx,NBy,Deg));
   #Convolution of matrices:
   for n in range(0,(Deg)):
       for m in range(0,n+1):
           if ((n-m)<NAz and m<NBz):
               C[:,:,n] = C[:,:,n]+ np.dot(A[:,:,(n-m)],B[:,:,m]);
   return C

In [ ]:
def MDCTanafb(x,N,fb):
   #MDCT analysis filter bank.
   #Arguments: x: input signal, e.g. audio signal, a 1-dim. array
   #N: number of subbands
   #fb: coefficients for the MDCT filter bank, for the F matrix, np.array with 1.5*N coefficients.
   #returns y, consisting of blocks of subband in in a 2-d array of shape (N,# of blocks)
   
   Fa=symFmatrix(fb)
   D=Dmatrix(N)
   y=x2polyphase(x,N)
   y=polmatmult(y,Fa)   
   y=polmatmult(y,D)
   y=DCT4(y)
   #strip first dimension:
   y=y[0,:,:]
   return y

y=MDCTanafb(x,576,fb)

In [ ]:
y.shape